In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()
import matplotlib.pyplot as plt
import numpy as np
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [31]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

class model:
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.X = tf.placeholder(tf.float32,[None,784])
            X_img = tf.reshape(self.X,[-1,28,28,1])
            self.Y = tf.placeholder(tf.float32,[None,10])
            self.keep_prob = tf.placeholder(tf.float32)
            W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev = 0.01))
            L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')
            L1 = tf.nn.relu(L1)
            L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
            L1 = tf.nn.dropout(L1,keep_prob=self.keep_prob)
            
            W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev = 0.01))
            L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
            L2 = tf.nn.relu(L2)
            L2 = tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
            L2 = tf.nn.dropout(L2,keep_prob=self.keep_prob)
            
            W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev = 0.01))
            L3 = tf.nn.conv2d(L2,W3,strides = [1,1,1,1],padding='SAME')
            L3 = tf.nn.relu(L3)
            L3 = tf.nn.max_pool(L3,ksize=[1,2,2,1],strides = [1,2,2,1],padding='SAME')
            L3 = tf.nn.dropout(L3,keep_prob=self.keep_prob)

            W4 = tf.Variable(tf.random_normal([3,3,128,256],stddev = 0.01))
            L4 = tf.nn.conv2d(L3,W4,strides = [1,1,1,1],padding='SAME')
            L4 = tf.nn.relu(L4)
            L4 = tf.nn.max_pool(L4,ksize=[1,2,2,1],strides = [1,2,2,1],padding='SAME')
            L4 = tf.reshape(L4,[-1,2*2*256])
            
            W5 = tf.Variable(tf.random_normal([2*2*256,10],stddev = 0.01))
            b = tf.Variable(tf.random_normal([10]))
            
            self.hypothesis = tf.matmul(L4,W5) +b
     
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.hypothesis, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(self.cost)
        correct_prediction = tf.equal(tf.argmax(self.hypothesis,1),tf.argmax(self.Y,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

            
    def train(self,x_data,y_data,keep_prob = 0.7):
        return self.sess.run([self.cost,self.optimizer],feed_dict = {self.X:x_data,self.Y:y_data,self.keep_prob : keep_prob})
   
    def predict(self,x_test,keep_prob = 0.1):
        return self.sess.run(self.hypothesis,feed_dict = {self.X:x_test,self.keep_prob : keep_prob})

    def get_accuracy(self,x_test,y_test,keep_prob=0.1):
        return self.sess.run(self.accuracy,feed_dict = {self.X:x_test,self.Y:y_test,self.keep_prob : keep_prob})




Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
sess = tf.Session()
m1 = model(sess,"m1")
sess.run(tf.global_variables_initializer())

print('learning started. It may take some time')
for epoch in range(15):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / 100)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        c,_ = m1.train(batch_xs,batch_ys)
        avg_cost += c/total_batch
    print('Epoch:','%04d'%(epoch+1),'cost','{:.9f}'.format(avg_cost))

print('learning finished')

learning started. It may take some time
Epoch: 0001 cost 0.582564103
Epoch: 0002 cost 0.113400076
Epoch: 0003 cost 0.081666905
Epoch: 0004 cost 0.063033858
Epoch: 0005 cost 0.053268993
Epoch: 0006 cost 0.046859243
Epoch: 0007 cost 0.044124991
Epoch: 0008 cost 0.038665857
Epoch: 0009 cost 0.034316766
Epoch: 0010 cost 0.032912995
Epoch: 0011 cost 0.030706438
Epoch: 0012 cost 0.027397258
Epoch: 0013 cost 0.027231312
Epoch: 0014 cost 0.025534249
